# CubeML prototyping

## Load libraries and get ready...

In [1]:
import pickle
import spectral
#spectral.settings.envi_support_nonlowercase_params = True
from cubeml import TrainingData
from cubeml import CubeSchool
from cubeml.model_evaluation import presentable_table
from cubeml.model_evaluation import generate_falsecolor_images
from cubeml.model_evaluation import compare_inferences
import os

ModuleNotFoundError: No module named 'cubeml'

In [ ]:
json_file = '../../03_Parse_labels_and_train/labels_a2_halogena1/project-2-at-2023-06-07-03-09-cee8e3b6.json'

hypercube_dir = "../../02_False_color_development/poplar_regen_halogen_a1/"

png_directory = "../../03_Parse_labels_and_train/labels_a2_halogen_png/"

In [ ]:
falsecolor_dir = "output/falsecolor/"
learner_plot_dir = "output/learner_plots/"
panel_image_dir = "output/panel_images/"

## Prepare models

In [ ]:
[os.makedirs(dir, exist_ok=True) for dir in [falsecolor_dir, learner_plot_dir, panel_image_dir]]

### Build `TrainingData`

In [ ]:
training_data = TrainingData(
    json_file=json_file,
    img_directory=hypercube_dir,
    png_directory=png_directory
)

training_data.stratified_sample()

filename = "output/" + os.path.basename(os.path.normpath(png_directory.replace('/', '_TrainingData.pkl')))
pickle.dump(training_data, open(filename, 'wb'))

In [ ]:
training_data.labels_key_df

In [ ]:
colors = ['#000000', '#0000FF', '#013220', '#FF0000', '#964B00']  # User-provided color list

### Build `CubeLearner` objects inside `CubeSchool`

In [ ]:
model_types = ["PCA", "LDA", "RF", "GBC", "ABC", "LR", "GNB", "DTC"]
data_types = ["Train", "Test"]

school = CubeSchool(training_data, model_types, colors)
school.run()

## Evaluation and deployment of `CubeLearner` objects

### Confusion matrix heatmaps and other model summary plots

In [ ]:
school.multi_plot(learner_plot_dir)

### IoU table

In [ ]:
table = presentable_table(training_data.labels_key_df, school.iou_dict)
filename = "output/" + os.path.basename(os.path.normpath(png_directory.replace('/', '_metrics.csv')))
table.save(filename)

### Inference plots

In [ ]:
df = cubeml.match_files(hypercube_dir)

In [ ]:
generate_falsecolor_images(df = df, 
                           learner_dict = school.learner_dict,
                           model_types = model_types,
                           colors = colors,
                           output_dir = falsecolor_dir, 
                           min_wave = 400, 
                           max_wave = 1000)

In [ ]:
compare_inferences(df = df,
                   model_types = model_types,
                   falsecolor_dir = falsecolor_dir,
                   output_dir = panel_image_dir,
                   num_panels = 3)

### `plot_spectra` for `TrainingData`

In [ ]:
fig = training_data.plot_spectra(colors, sampling_fraction = 0.001)
spectra_filename = png_directory.replace('/', '_spectra.png')
fig.savefig("output/" + spectra_filename)
fig.show()